In [1]:
import syft as sy
import numpy as np
from syft import phi_tensor

from syft.core.adp.data_subject_ledger import DataSubjectLedger
from syft.core.adp.ledger_store import DictLedgerStore
from typing import Any
from syft.core.adp.data_subject import DataSubject
import numpy as np

data1 = np.random.random((5,5))
data2 = np.random.random((5,5))

pt = phi_tensor.PhiTensor(child=data1, data_subjects=1, min_vals=0, max_vals=1)
pt2 = phi_tensor.PhiTensor(child=data2, data_subjects=2, min_vals=0, max_vals=1)
gt = pt + pt2
gt.state.keys(), gt.data_subjects.data_subjects_indexed

/Users/ivolinengong/miniforge3/envs/syft_env/lib/python3.10/site-packages/jax/_src/lib/__init__.py:33: UserWarning: JAX on Mac ARM machines is experimental and minimally tested. Please see https://github.com/google/jax/issues/5501 in the event of problems.
  warnings.warn("JAX on Mac ARM machines is experimental and minimally tested. "
/Users/ivolinengong/miniforge3/envs/syft_env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(dict_keys(['115251055', '1875872415']), array(0))

In [2]:
ledger_store = DictLedgerStore()
# print(ledger_store.kv_store)
user_key = b"1231"
ledger = DataSubjectLedger.get_or_create(store=ledger_store, user_key=user_key)
max_budget = 100

def get_budget_for_user(*args: Any, **kwargs: Any) -> float:
    return 9999999

def deduct_epsilon_for_user(*args: Any, **kwargs: Any) -> bool:
    return True


Creating new Ledger


Call publish

Set calibration==false and true

verify that budget spent is actually less than the query limit

syftensor=>phi_tensor
syftensor=>gamma_tensor

In [3]:

# Phi Tensor
results = pt.publish(
    get_budget_for_user=get_budget_for_user,
    deduct_epsilon_for_user=deduct_epsilon_for_user,
    ledger=ledger,
    sigma=0.1,
)



PUBLISHING TO GAMMA:
FixedPrecisionTensor(child=[[39632 48600 59897 23518 24331]
 [18162 51897 60692 62180 19321]
 [25920 50573 14825 59426 14750]
 [47556 19305  2533 61419 43769]
 [53502 56616 38664 21280 44793]])
****** Budget spent EXCEEDS limit set by data owner


  0%|          | 0/1 [00:00<?, ?it/s]

constant:  [9.73900626 9.73900626 9.73900626 9.73900626 9.73900626 9.73900626
 9.73900626 9.73900626 9.73900626 9.73900626 9.73900626 9.73900626
 9.73900626 9.73900626 9.73900626 9.73900626 9.73900626 9.73900626
 9.73900626 9.73900626 9.73900626 9.73900626 9.73900626 9.73900626
 9.73900626]
_rdp_constants:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]
entity ids query 0


/Users/ivolinengong/miniforge3/envs/syft_env/lib/python3.10/site-packages/jax/interpreters/mlir.py:512: UserWarning: Some donated buffers were not usable: ShapedArray(float64[25]), ShapedArray(int64[]).
Donation is not implemented for cpu.
See an explanation at https://jax.readthedocs.io/en/latest/faq.html#buffer-donation.
  warnings.warn(f"Some donated buffers were not usable: {', '.join(unused_donations)}.\n{msg}")
100%|██████████| 1/1 [00:00<00:00,  9.69it/s]

0
Epsilon spend  [29.47263845 29.47263845 29.47263845 29.47263845 29.47263845 29.47263845
 29.47263845 29.47263845 29.47263845 29.47263845 29.47263845 29.47263845
 29.47263845 29.47263845 29.47263845 29.47263845 29.47263845 29.47263845
 29.47263845 29.47263845 29.47263845 29.47263845 29.47263845 29.47263845
 29.47263845]
Highest possible spend  29.472638447378344
Attemping to spend epsilon: 29.472638447378344. Try: 0
got user budget 9999999.0 epsilon_spent 29.472638447378344
We have filtered all the input tensors. Now to compute the result:
Filtered inputs  [array([[39632, 48600, 59897, 23518, 24331],
       [18162, 51897, 60692, 62180, 19321],
       [25920, 50573, 14825, 59426, 14750],
       [47556, 19305,  2533, 61419, 43769],
       [53502, 56616, 38664, 21280, 44793]])]
original output (before noise: [[[39632 48600 59897 23518 24331]
  [18162 51897 60692 62180 19321]
  [25920 50573 14825 59426 14750]
  [47556 19305  2533 61419 43769]
  [53502 56616 38664 21280 44793]]]
noise:  [[

In [4]:
# Gamma Tensor
results = gt.publish(
    get_budget_for_user=get_budget_for_user,
    deduct_epsilon_for_user=deduct_epsilon_for_user,
    ledger=ledger,
    sigma=0.1,
)

****** Budget spent EXCEEDS limit set by data owner


100%|██████████| 2/2 [00:00<00:00, 780.63it/s]

constant:  [9.66459041 9.66459041 9.66459041 9.66459041 9.66459041 9.66459041
 9.66459041 9.66459041 9.66459041 9.66459041 9.66459041 9.66459041
 9.66459041 9.66459041 9.66459041 9.66459041 9.66459041 9.66459041
 9.66459041 9.66459041 9.66459041 9.66459041 9.66459041 9.66459041
 9.66459041]
_rdp_constants:  [9.73900626 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.        ]
entity ids query 0
0
Epsilon spend  [29.30702117 29.30702117 29.30702117 29.30702117 29.30702117 29.30702117
 29.30702117 29.30702117 29.30702117 29.30702117 29.30702117 29.30702117
 29.30702117 29.30702117 29.30702117 29.30702117 29.30702117 29.30702117
 29.30702117 29.30702117 29.30702117 29.30702117 29.30702117 29.30702117
 29.30702117]
Highest possible spend  29.307021168236922
Attemping to spend epsilon: 29.307021168236922. Try: 0
go

Syft Tensors

In [5]:

entity1 = np.random.choice([DataSubject(name="1")], size=5)
entity2 = np.random.choice( [DataSubject(name="2")], size=5)

pt_tensor = sy.Tensor(np.random.random((5,5))).private(min_val=0, max_val=2, data_subjects=entity1)
pt_tensor2 = sy.Tensor(np.random.random((5,5))).private(min_val=0, max_val=2, data_subjects=entity2)
pt_tensor.shape, pt.shape

((5, 5), (5, 5))

In [6]:
# Gamma Tensor
results = pt_tensor.child.publish(
    get_budget_for_user=get_budget_for_user,
    deduct_epsilon_for_user=deduct_epsilon_for_user,
    ledger=ledger,
    sigma=0.1,
)

PUBLISHING TO GAMMA:
FixedPrecisionTensor(child=[[56546  6618 18636 14517 36664]
 [ 4828   931 45263 18288  7290]
 [39617 22252  5839 15680 62367]
 [ 2606 19209 15301  3784 65030]
 [25696 58126  2399 64349  5447]])
****** Budget spent EXCEEDS limit set by data owner


  0%|          | 0/1 [00:00<?, ?it/s]/Users/ivolinengong/miniforge3/envs/syft_env/lib/python3.10/site-packages/jax/interpreters/mlir.py:512: UserWarning: Some donated buffers were not usable: ShapedArray(float64[25]), ShapedArray(int64[5]).
Donation is not implemented for cpu.
See an explanation at https://jax.readthedocs.io/en/latest/faq.html#buffer-donation.
  warnings.warn(f"Some donated buffers were not usable: {', '.join(unused_donations)}.\n{msg}")
100%|██████████| 1/1 [00:00<00:00, 25.21it/s]

constant:  [5.76310831 5.76310831 5.76310831 5.76310831 5.76310831 5.76310831
 5.76310831 5.76310831 5.76310831 5.76310831 5.76310831 5.76310831
 5.76310831 5.76310831 5.76310831 5.76310831 5.76310831 5.76310831
 5.76310831 5.76310831 5.76310831 5.76310831 5.76310831 5.76310831
 5.76310831]
_rdp_constants:  [27.95726202  0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.        ]
entity ids query [0 0 0 0 0]
0
Epsilon spend  [19.93090564 19.93090564 19.93090564 19.93090564 19.93090564 19.93090564
 19.93090564 19.93090564 19.93090564 19.93090564 19.93090564 19.93090564
 19.93090564 19.93090564 19.93090564 19.93090564 19.93090564 19.93090564
 19.93090564 19.93090564 19.93090564 19.93090564 19.93090564 19.93090564
 19.93090564]
Highest possible spend  19.930905642716098
Attemping to spend eps

In [7]:
# Gamma Tensor
gt_tensor = pt_tensor + pt_tensor2
results = gt_tensor.child.publish(
    get_budget_for_user=get_budget_for_user,
    deduct_epsilon_for_user=deduct_epsilon_for_user,
    ledger=ledger,
    sigma=0.1,
)

****** Budget spent EXCEEDS limit set by data owner


100%|██████████| 2/2 [00:00<00:00, 716.36it/s]

constant:  [6.00073617 6.00073617 6.00073617 6.00073617 6.00073617 6.00073617
 6.00073617 6.00073617 6.00073617 6.00073617 6.00073617 6.00073617
 6.00073617 6.00073617 6.00073617 6.00073617 6.00073617 6.00073617
 6.00073617 6.00073617 6.00073617 6.00073617 6.00073617 6.00073617
 6.00073617]
_rdp_constants:  [33.72037033  0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.        ]
entity ids query [0 0 0 0 0]
0
Epsilon spend  [20.55403169 20.55403169 20.55403169 20.55403169 20.55403169 20.55403169
 20.55403169 20.55403169 20.55403169 20.55403169 20.55403169 20.55403169
 20.55403169 20.55403169 20.55403169 20.55403169 20.55403169 20.55403169
 20.55403169 20.55403169 20.55403169 20.55403169 20.55403169 20.55403169
 20.55403169]
Highest possible spend  20.554031690244358
Attemping to spend eps